In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from statsmodels.stats.multitest import multipletests


In [ ]:
#Загружаем данные

installs = pd.read_csv('data/installs.csv')
logins = pd.read_csv('data/logins.csv')
payments = pd.read_csv('data/payments.csv')


# Описание данных

In [ ]:
#Проверяем на пропуски
installs.info()
logins.info()
payments.info()

#Пропусков в данных нет

In [ ]:
# Посмотрим на доступные к покупке опции для каждой группы
# Покупки контрольной группы
payments[ payments.group == 1 ].groupby('payment_tag')['price'].agg('unique')

In [ ]:
# Покупки группы 2
payments[ payments.group == 2 ].groupby('payment_tag')['price'].agg('unique')

In [ ]:
# Покупки группы 2
payments[ payments.group == 3 ].groupby('payment_tag')['price'].agg('unique')

In [ ]:
# Установки, логины, покупки

print(
    f'Количество уникальных установок:{installs.user_id.nunique()}, \n'
    f'Количество уникальных логинов: {logins.user_id.nunique()}, \n'
    f'Количество уникальных покупателей: {payments.user_id.nunique()}'

)

In [ ]:
# Доход всего

print(
    f'Доход всего: ${payments.price.sum()}\n'
)

In [ ]:
# Доход по группам
pd.DataFrame(payments.groupby('group')['price'].sum())

In [ ]:
# Перед объединением данных и проведением тестов проверим корректность данных
# - каждый пользователь должен находиться только в одной группе

installs[ installs.user_id.duplicated(keep=False) ==True ]
# У каждого пользователя только одна установка

In [ ]:
# Пользователи в датасете logins, записанные несколько раз, должны относяться только к одной группе.
# Появление нескольких записей нормально - пользователи логинятся несколько раз, но нужно проверить нет ли ошибки
# - не отнесены ли пользователи к разным группам.
logins[ logins.user_id.duplicated(keep=False) ==True ].\
    groupby('user_id').\
    agg({'group':'unique'}).\
    sort_values(ascending=False, by='group')

# Каждый пользователь относится только к одной группе

In [ ]:
# Пользователи в датасете payments, записанные несколько раз, должны относяться только к одной группе
# С данными о покупке тоже самое - пользователи могут покупать несколько раз, но нужно проверить нет ли ошибки
# - не отнесены ли пользователи к разным группам
payments[ payments.user_id.duplicated(keep=False) ==True ].\
    groupby('user_id').\
    agg({'group':'unique'}).\
    sort_values(ascending=False, by='group')

# Каждый пользователь относится только к одной группе

Все пользователи относятся только к одной группе, в данных нет ошибок

In [ ]:
# Длительность теста
# Привести все таймстампы к дейт тайму
installs['datetime'] = pd.to_datetime(installs['timestamp'], unit='s')
logins['datetime'] = pd.to_datetime(logins['timestamp'], unit='s')
payments['datetime'] = pd.to_datetime(payments['timestamp'], unit='s')

print(
    f'Длительность A/B/C-теста: {logins.datetime.max()-logins.datetime.min()}'
)



# Описание данных:
Есть 3 тестовые группы:
- контрольная,
- облегченный стартовый набор, стоимость ниже обычной в 5 раз, наполнение в 2 раза
- более выгодные daily supplies - отдельные наборы ресурсов (3 вида), получение части покупки происходит в течение 30 дней
___________________
**Контрольная группа**

Стартер пак, $4.99

Обычные наборы, $4.99, $9.99, $19.99, $49.99, $99.99, $249.99

___________________
**Группа 2**
Облегченный стартер пак, $0.99

Обычные наборы, $4.99, $9.99, $19.99, $49.99, $99.99, $249.99

___________________
**Группа 3**
Стартер пак, $4.99

Daily supplies, $9.99, $19.99

Обычные наборы, $4.99, $9.99, $19.99, $49.99, $99.99, $249.99

___________________

Скорее всего тестируется несколько гипотез:
- Чем руководствуются пользователи: дешевизна или выгода?
- Станут ли пользователи чаще конвертироваться в покупку, если снизить порог входа? (группа 2)
- Будут ли пользователи, которые совершили небольшую первую покупку затем играть и тратить столько же, сколько и другие? (группа 2)
- Как на пользователей влияет фактор длительности получения покупки? (группа 3)
- Нужны ли пользователям отдельные наборы ресурсов? (группа 3)



# Расчет метрик


In [ ]:
# Оставляем только записи с первыми логинами

# Группируем датасет logins по user_id и group, выбираем минимальную и максимальную дату для каждого юзера
first_logins = logins.groupby(['user_id', 'group'])\
     .agg({'datetime':['min', 'max']}).reset_index(col_level=1)

# Убираем многоуровневость из названий колонок
first_logins.columns = first_logins.columns.droplevel(level=0)

# Переименовываем
first_logins.rename(columns={'min': 'datetime_first_login', 'max':'datetime_last_login'}, inplace=True)
first_logins.head()



In [ ]:
# Добавляем к логинам все платежи

# Объединяем датасет logins с payments
first_logins_payments = first_logins.merge(
    payments[['user_id', 'datetime', 'payment_tag', 'price']],
    how='left',
    on='user_id',
)

# Переименовываем
first_logins_payments.rename(columns={'datetime': 'datetime_payment'}, inplace=True)
first_logins_payments.head(50)

#Каждый пользователь продублирован по количеству платежей

# ARPU

In [ ]:
# Считаем ARPU и ARPPU по тестовым группам
#Заполняем null на 0
first_logins_payments.fillna(value={'price':0}, inplace=True)

# Группируем по тестовым группым и считаем метрики
arpu_per_group = first_logins_payments.groupby('group')\
    .agg(
    {
        'price': [sum],
        'user_id': ['nunique']
    }
)

# Убираем многоуровневость из колонок, переименовываем колонки
arpu_per_group.columns = arpu_per_group.columns.droplevel(level=0)
arpu_per_group.rename(columns={'nunique': 'unique_users', 'sum':'revenue'}, inplace=True)
arpu_per_group['arpu'] = arpu_per_group.revenue/arpu_per_group.unique_users
arpu_per_group

In [ ]:
# Рассчитаем минимальный размер выборки для теста arpu, чтобы убедиться, что данных достаточно
import pingouin as pg
from numpy import ceil
#Считаем effect size для трех групп
aov = pg.anova(
    dv='price',
    between='group',
    data=first_logins_payments,
    detailed=True,
    effsize='n2'
)

#Считаем минимально необходимый размер групп
group_size = pg.power_anova(
    eta=aov.n2[0],
    k=3, power=0.80,
    alpha=0.05
)
print(
    f'Минимальный необходимый размер групп:{ceil(group_size/3)}'
)
print('Количество пользователей в группах:')
first_logins_payments.groupby('group').user_id.nunique()

# Размер выборки достаточный

### Какой провести тест?
Необходимо сравнить распределение покупок в 3 группах.

Fisher f-test условия:
- Нормальное распределение
- Независимость данных

Tuckey HSD test условия:
- Независимость данных
- Нормальное распределение
- Одинаковая дисперсия в выборках

Kruskal-wallis test условия:
- Зависимая переменная - порядковая или непрерывная
- Независимость данных
- Все группы должны иметь одинаковую форму распределения (использует медианы для расчета)
- Не очень хорошо работает с повторами


In [ ]:
#Проводим проверку на нормальность распределения
from stat_funcs import visual_normality_check, statistical_normality_test

control = first_logins_payments[first_logins_payments.group==1]
group2 = first_logins_payments[first_logins_payments.group==2]
group3 = first_logins_payments[first_logins_payments.group==3]

groups = {'control group':control, 'test group 2':group2, 'test group 3':group3}
for group_name, group_df in groups.items():
    print(f'Check normality for {group_name}')
    visual_normality_check(feature_name='price', df_name=group_df)
    statistical_normality_test(feature_name='price', df_name=group_df, alpha=0.05)

#Все распределения не нормальные

In [ ]:
# Делаем проверку на гомогенность дисперсий
from stat_funcs import equal_var_test

equal_var_test(control.price, group2.price, group3.price)

# Дисперсии не гомогенны

In [ ]:
# Проверим одинакова ли форма распределений

fig, ax = plt.subplots(figsize=(16,10))
sns.kdeplot(control.price)
sns.kdeplot(group2.price)
sns.kdeplot(group3.price)
ax.legend(['control', 'group2', 'group3'])
plt.show()

#Форма распределений практически идентична

Fisher и Tuckey не подходят, т.к. не соблюдены условия нормальности распределения и гомогенности дисперсий.

Критерий Краскела-Уоллиса плохо работают с повторяющимися значениями,
но в библиотеке scipy используются методы корректировки для повторяющихся значений.


### Kruskal-Wallis test

In [ ]:
from scipy import stats
kr_val, p_val = stats.kruskal(np.array(control.price), np.array(group2.price), np.array(group3.price))
print(
    f'Kruskall-Wallis test \n'
    f'Statistics={np.round(kr_val, 3)}, p-value={p_val} \n'
)
alpha = 0.05
if p_val > alpha:
    print('Kruskall-Wallis: Samples have equal medians (fail to reject H0) \n')
else:
    print('Kruskall-Wallis: Samples medians are not equal (can reject H0) \n')

Мы установили наличие статистически значимых различий, но тест Краскелла-Уоллиса не дает информацию о том,
какие именно группы различаются. Для этого нужно провести post hoc (апосетриорный) тест.
Используем Dunn's test (чаще всего используют после теста Краскелла-Уоллиса).

In [ ]:
import scikit_posthocs as sp

x = [np.array(control.price), np.array(group2.price), np.array(group3.price)]
dunn_test = sp.posthoc_dunn(x, p_adjust = 'holm')
dunn_test = pd.DataFrame(dunn_test.stack()).drop_duplicates(subset=0)[1:].rename(columns={0:'p_value'}).reset_index()
dunn_test['pairs'] = dunn_test['level_0'].astype(str) + '-' + dunn_test['level_1'].astype(str)
dunn_test = dunn_test[['pairs', 'p_value']]
dunn_test
#Все пары значимо различаются

In [ ]:
#Создадим датасет с разницей в ARPU между группами
arpu_diff = pd.DataFrame(
    [arpu_per_group.arpu[1] - arpu_per_group.arpu[2],
    arpu_per_group.arpu[1] - arpu_per_group.arpu[3],
    arpu_per_group.arpu[2] - arpu_per_group.arpu[3]],
    columns=['arpu_difference'],
    index=['1-2', '1-3',  '2-3']
)

# Объединим результаты теста с разницей между группами, чтобы посмотреть направление и размер эффекта
arpu_test = dunn_test.merge(
    arpu_diff,
    how='inner',
    left_on='pairs',
    right_on=arpu_diff.index
)
arpu_test['reject'] = (arpu_test.p_value<0.05)
arpu_test

Вывод: разница ARPU всех трех групп статистически значима.
Лучший результат показала группа 3, худший - группа 2.

# Conversion rate to payment

In [ ]:
# Рассчитаем значение CR

payments_per_user = first_logins_payments.\
    groupby(['user_id', 'group', 'datetime_first_login', 'datetime_last_login']).\
    agg({'price': 'sum'}).reset_index()

CR_per_group = payments_per_user.groupby('group')\
    .agg(
    {
        'user_id': 'nunique',
        'price': lambda x: (x>0).sum(),
    }
)

CR_per_group['conv_rate'] = 100*(CR_per_group['price']/CR_per_group['user_id'])
CR_per_group.rename(columns={'user_id':'all_users', 'price':'converted_users'}, inplace=True)
CR_per_group

Для расчета статистической значимости конверсий можно использовать:
- нормализацию и z-test, метод Бонферрони-Холма для правки уровня значимости при множественных сравнениях

## Z-test for CR

Z-test условия:
- Данные в генеральной совокупности распределены нормально
- Размер выборки n>30

In [ ]:
# Добавим колонку сконвертировался ли юзер в оплату, для дальнейших расчетов
payments_per_user['is_converted'] = payments_per_user.price>0.0
payments_per_user

In [ ]:
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

# Оставляем только колонку - сконвертировался ли пользователь
control_results = payments_per_user[payments_per_user['group'] == 1]['is_converted']
group2_results = payments_per_user[payments_per_user['group'] == 2]['is_converted']
group3_results = payments_per_user[payments_per_user['group'] == 3]['is_converted']

# Считаем количество всех пользователей
n_con = control_results.count()
n_group2 = group2_results.count()
n_group3 = group3_results.count()

# Количество сконвертировавшихся пользователей
successes = [control_results.sum(), group2_results.sum(), group3_results.sum()]
nobs = [n_con, n_group2, n_group3]

In [ ]:
# Посчитаем доверительные интервалы
(lower_con, lower_group2, lower_group3), (upper_con, upper_group2, upper_group3) = \
    proportion_confint(successes, nobs=nobs, alpha=0.05)

# Доверительные интервалы
print(f'ci 95% for control group: [{lower_con:.3f}, {upper_con:.3f}]')
print(f'ci 95% for treatment 2nd group: [{lower_group2:.3f}, {upper_group2:.3f}]')
print(f'ci 95% for treatment 3rd group: [{lower_group3:.3f}, {upper_group3:.3f}]')

# Скорее всего, значимые отличия есть только для второй группы - ее доверительный интервал не пересекается с другими

In [ ]:
# Proportion z-test для control и group 2

z_stat1, p_val1 = proportions_ztest(
    [control_results.sum(), group2_results.sum()],
    nobs=[n_con, n_group2]
)
print(f'z statistic: {z_stat1:.2f}')
print(f'p-value: {p_val1:.3f}')

alpha = 0.05
if p_val > alpha:
    print('Z-test: Samples have equal means (fail to reject H0) \n')
else:
    print('Z-test: Samples means are not equal (can reject H0) \n')

In [ ]:
# Proportion z-test для control и group 3

z_stat2, p_val2 = proportions_ztest(
    [control_results.sum(), group3_results.sum()],
    nobs=[n_con, n_group3]
)
print(f'z statistic: {z_stat2:.2f}')
print(f'p-value: {p_val2:.3f}')

alpha = 0.05
if p_val2 > alpha:
    print('Z-test: Samples have equal means (fail to reject H0) \n')
else:
    print('Z-test: Samples means are not equal (can reject H0) \n')

In [ ]:
# test group2, group3
z_stat3, p_val3 = proportions_ztest(
    [group2_results.sum(), group3_results.sum()],
    nobs=[n_group2, n_group3]
)
print(f'z statistic: {z_stat3:.2f}')
print(f'p-value: {p_val3:.5f}')

alpha = 0.05
if p_val3 > alpha:
    print('Z-test: Samples have equal means (fail to reject H0) \n')
else:
    print('Z-test: Samples means are not equal (can reject H0) \n')

In [ ]:
# Добавим результаты в один датасет
CR_z_tets = pd.DataFrame(np.array([
    ['1-2', z_stat1, p_val1],
    ['1-3', z_stat2, p_val2],
    ['2-3', z_stat3, p_val3]
]),
    columns=['pairs', 'z_stat', 'p_value'])
CR_z_tets[['z_stat', 'p_value']] = CR_z_tets[['z_stat', 'p_value']].apply(pd.to_numeric)
CR_z_tets

In [ ]:
# Делаем поправку на множественное сравнение
res = multipletests(pvals=CR_z_tets.p_value, alpha=0.05, method='h')
print(res[0], res[1])
CR_z_tets['adjusted_p'] = res[1]
CR_z_tets['reject'] = res[0]
CR_z_tets

Вторая группа значимо лучше конвертирует в оплату, размер эффекта ~ 0,4%.
Т.е. разница в 4 человека с 1000 по сравнению с другими группами.
Между контрольной и третьей группой статистически значимых различий нет.

# Retention 30

In [ ]:
# Рассчитать метрики

# Добавим к логинам информацию сконвертировался ли пользователь в покупки или нет
logins_conv = logins.merge(
    payments_per_user[['user_id', 'is_converted', 'datetime_first_login', 'datetime_last_login']],
    how='left',
    on='user_id'
)
# Меняем тип данных - убираем время и оставляем только дату (для расчета retention)
logins_conv['datetime_first_login'] = logins_conv['datetime_first_login'].dt.date
logins_conv['datetime_last_login'] = logins_conv['datetime_last_login'].dt.date
logins_conv['datetime'] = logins_conv['datetime'].dt.date
logins_conv

# Добавляем колонку seniority с "возрастом" пользователя в днях
logins_conv['seniority']= (logins_conv['datetime'] - logins_conv['datetime_first_login']).dt.days
logins_conv

In [ ]:
#Группируем записи по дате первого логина и "возрасту", считаем количество записей
cohort_data = logins_conv.groupby(['datetime_first_login', 'seniority'])['user_id'].size().reset_index()

# Разворачиваем таблицу, индекс-когорты, колонки-"возраст"
cohorts_size = cohort_data.pivot(index='datetime_first_login', columns='seniority', values='user_id')

# Выделяем 0 день, чтобы с помощью него посчитать конверсии
base = cohorts_size[0]
cohorts_retention = cohorts_size.divide(base, axis=0).round(3)*100
cohorts_retention.style.background_gradient()

In [ ]:
# Посмотрим ретеншен по группам, а не по когортам

# Оставим только нужные колонки и уберем дубликаты
logins_groups =  logins_conv[['group', 'seniority', 'user_id']].drop_duplicates()

# Группируем записи по тестовой группе и "возрасту", считаем количество записей
group_data = logins_groups.groupby(['group', 'seniority'])['user_id'].size().reset_index()

# Разворачиваем таблицу, индекс-группы, колонки-"возраст"
groups_size = group_data.pivot(index='group', columns='seniority', values='user_id')
base = groups_size[0]
groups_retention = groups_size.divide(base, axis=0).round(5)*100

# Выделим ячейки цветом по "возрасту"
groups_retention.style.background_gradient()

In [ ]:
# Сохраняем retention 1, 7, 30 в отдельный датасет
retention_1_7_30 = groups_retention[[1, 7, 30]]
retention_1_7_30.rename(columns={1:'retention_1', 7:'retention_7', 30:'retention_30'}, inplace=True)
retention_1_7_30

In [ ]:
# Добавляем поле returned_30 - вернулся ли пользователь на 30 день
logins_conv['returned_30'] = (logins_conv.datetime_last_login - logins_conv.datetime_first_login).\
                                 dt.days.astype('int16') >= 30
# Оставляем только одну запись на пользователя
logins_unique = \
    logins_conv[['user_id', 'group', 'datetime_first_login', 'datetime_last_login', 'is_converted', 'returned_30']].\
    drop_duplicates()
logins_unique

In [ ]:
# - делим на 3 датасета по группам
retention_group1 = logins_unique[ logins_unique.group == 1 ][[ 'user_id', 'group', 'is_converted', 'returned_30' ]]
retention_group2 = logins_unique[ logins_unique.group == 2 ][[ 'user_id', 'group', 'is_converted', 'returned_30' ]]
retention_group3 = logins_unique[ logins_unique.group == 3 ][[ 'user_id', 'group', 'is_converted', 'returned_30' ]]

In [ ]:
# Считаем количество записей в группах и количество "успехов" - Ture
r_group1_n = retention_group1['user_id'].nunique()
r_group2_n = retention_group2['user_id'].nunique()
r_group3_n = retention_group3['user_id'].nunique()

r_group1_successes = retention_group1['returned_30'].sum()
r_group2_successes = retention_group2['returned_30'].sum()
r_group3_successes = retention_group3['returned_30'].sum()

nobs = [r_group1_n, r_group2_n, r_group3_n]
successes = [r_group1_successes, r_group2_successes, r_group3_successes]

In [ ]:
# Считаем значения доверительных интервалов для каждой выборки
(lower_con, lower_group2, lower_group3), (upper_con, upper_group2, upper_group3) = \
    proportion_confint(successes, nobs=nobs, alpha=0.05)
print(f'ci 95% for control group: [{lower_con:.5f}, {upper_con:.5f}]')
print(f'ci 95% for treatment 2nd group: [{lower_group2:.5f}, {upper_group2:.5f}]')
print(f'ci 95% for treatment 3rd group: [{lower_group3:.5f}, {upper_group3:.5f}]')

# Явной разницы между группами нет - все доверительные интервалы пересекаются

In [ ]:
# Z-тест test для retention 30 control и group 2
z_stat1, p_val1 = proportions_ztest(
    [r_group1_successes.sum(), r_group2_successes.sum()],
    nobs=[r_group1_n, r_group2_n]
)
print(f'z statistic: {z_stat1:.5f}')
print(f'p-value: {p_val1:.5f}')

alpha = 0.05
if p_val1 > alpha:
    print('Z-test: Samples have equal means (fail to reject H0) \n')
else:
    print('Z-test: Samples means are not equal (can reject H0) \n')

In [ ]:
# Z-тест test для retention 30 control, group 3
z_stat2, p_val2 = proportions_ztest(
    [r_group1_successes.sum(), r_group3_successes.sum()],
    nobs=[r_group1_n, r_group3_n]
)
print(f'z statistic: {z_stat2:.5f}')
print(f'p-value: {p_val2:.5f}')

alpha = 0.05
if p_val2 > alpha:
    print('Z-test: Samples have equal means (fail to reject H0) \n')
else:
    print('Z-test: Samples means are not equal (can reject H0) \n')

In [ ]:
# Z-тест test для retention 30 group2, group3
z_stat3, p_val3 = proportions_ztest(
    [r_group2_successes.sum(), r_group3_successes.sum()],
    nobs=[r_group2_n, r_group3_n]
)
print(f'z statistic: {z_stat3:.5f}')
print(f'p-value: {p_val3:.5f}')

alpha = 0.05
if p_val3 > alpha:
    print('Z-test: Samples have equal means (fail to reject H0) \n')
else:
    print('Z-test: Samples means are not equal (can reject H0) \n')


In [ ]:
# Делаем поправку на множественные сравнения
z_test_retention_res = pd.DataFrame(np.array([
    ['1-2', z_stat1, p_val1],
    ['1-3', z_stat2, p_val2],
    ['2-3', z_stat3, p_val3]
]),
    columns=['pairs', 'z_stat', 'p_value'])

z_test_retention_res[['z_stat', 'p_value']] = z_test_retention_res[['z_stat', 'p_value']].apply(pd.to_numeric)

res = multipletests(pvals=z_test_retention_res.p_value, alpha=0.05, method='hs')
print(res[0], res[1])
z_test_retention_res['adjusted_p'] = res[1]
z_test_retention_res['reject'] = res[0]
z_test_retention_res

# Статистической разницы в retention 30 дня между группами для всех пользователей - нет
# В группах все пользователи - совершившие покупки

In [ ]:
# Проверим есть ли разница в возвращаемости между группами пользователей совершивших покупку
logins_unique_conv = logins_conv[['user_id', 'group', 'is_converted', 'returned_30', 'seniority']].drop_duplicates()
logins_unique_conv = logins_unique_conv[logins_unique_conv.is_converted==True]

# Группируем записи по тестовой группе и "возрасту", считаем количество записей
group_data_conv = logins_unique_conv.groupby(['group', 'seniority'])['user_id'].size().reset_index()

# Разворачиваем таблицу, индекс-группы, колонки-"возраст"
groups_size_conv = group_data_conv.pivot(index='group', columns='seniority', values='user_id')
base = groups_size_conv[0]
groups_retention_conv = groups_size_conv.divide(base, axis=0).round(5)*100
groups_retention_conv.style.background_gradient()


In [ ]:
# Сохраняем retention 1, 7, 30 в отедельный датасет
retention_conv_1_7_30 = groups_retention_conv[[1, 7, 30]]
retention_conv_1_7_30.rename(columns={1:'retention_1', 7:'retention_7', 30:'retention_30'}, inplace=True)
retention_conv_1_7_30

In [ ]:
# делим на 3 датасета по группам
retention_conv_group1 = logins_unique_conv[ logins_unique_conv.group == 1 ][[ 'user_id', 'group', 'is_converted', 'returned_30' ]].drop_duplicates()
retention_conv_group2 = logins_unique_conv[ logins_unique_conv.group == 2 ][[ 'user_id', 'group', 'is_converted', 'returned_30' ]].drop_duplicates()
retention_conv_group3 = logins_unique_conv[ logins_unique_conv.group == 3 ][[ 'user_id', 'group', 'is_converted', 'returned_30' ]].drop_duplicates()

In [ ]:
# - считаем количество записей в группах, количество "успехов" - Ture
r_conv_group1_n = retention_conv_group1['user_id'].nunique()
r_conv_group2_n = retention_conv_group2['user_id'].nunique()
r_conv_group3_n = retention_conv_group3['user_id'].nunique()

r_conv_group1_successes = retention_conv_group1['returned_30'].sum()
r_conv_group2_successes = retention_conv_group2['returned_30'].sum()
r_conv_group3_successes = retention_conv_group3['returned_30'].sum()

nobs = [r_conv_group1_n, r_conv_group2_n, r_conv_group3_n]
successes = [r_conv_group1_successes, r_conv_group2_successes, r_conv_group3_successes]

In [ ]:
# Cчитаем значения доверительных интервалов для каждой выборки
(lower_con, lower_group2, lower_group3), (upper_con, upper_group2, upper_group3) = \
    proportion_confint(successes, nobs=nobs, alpha=0.05)

# Доверительный интервал
print(f'ci 95% for control group: [{lower_con:.5f}, {upper_con:.5f}]')
print(f'ci 95% for treatment 2nd group: [{lower_group2:.5f}, {upper_group2:.5f}]')
print(f'ci 95% for treatment 3rd group: [{lower_group3:.5f}, {upper_group3:.5f}]')

# Группа 2 скорее всего значимо отличается от остальных групп
# - доверительный интервал 2 группы не пересекается с другими

In [ ]:
# z-тест для каждой группы

# Z-тест test для retention 30 для test control, group2 (пользователи совершившие покупку)
z_stat1, p_val1 = proportions_ztest(
    [r_conv_group1_successes.sum(), r_conv_group2_successes.sum()],
    nobs=[r_conv_group1_n, r_conv_group2_n]
)
print(f'z statistic: {z_stat1:.5f}')
print(f'p-value: {p_val1:.5f}')

alpha = 0.05
if p_val1 > alpha:
    print('Z-test: Samples have equal means (fail to reject H0) \n')
else:
    print('Z-test: Samples means are not equal (can reject H0) \n')

In [ ]:
# Z-тест test для retention 30 для test control, group3 (пользователи совершившие покупку)

z_stat2, p_val2 = proportions_ztest(
    [r_conv_group1_successes.sum(), r_conv_group3_successes.sum()],
    nobs=[r_conv_group1_n, r_conv_group3_n]
)
print(f'z statistic: {z_stat2:.5f}')
print(f'p-value: {p_val2:.5f}')

alpha = 0.05
if p_val2 > alpha:
    print('Z-test: Samples have equal means (fail to reject H0) \n')
else:
    print('Z-test: Samples means are not equal (can reject H0) \n')

In [ ]:
# Z-тест test для retention 30 для test group2, group3 (пользователи совершившие покупку)

z_stat3, p_val3 = proportions_ztest(
    [r_conv_group2_successes.sum(), r_conv_group3_successes.sum()],
    nobs=[r_conv_group2_n, r_conv_group3_n]
)
print(f'z statistic: {z_stat3:.5f}')
print(f'p-value: {p_val3:.5f}')

alpha = 0.05
if p_val3 > alpha:
    print('Z-test: Samples have equal means (fail to reject H0) \n')
else:
    print('Z-test: Samples means are not equal (can reject H0) \n')

In [ ]:
# Делаем поправку на множественное сравнение
z_test_retention_conv_res = pd.DataFrame(np.array([
    ['paying_1-2', z_stat1, p_val1],
    ['paying_1-3', z_stat2, p_val2],
    ['paying_2-3', z_stat3, p_val3]
]),
    columns=['pairs', 'z_stat', 'p_value'])

z_test_retention_conv_res[['z_stat', 'p_value']] = z_test_retention_conv_res[['z_stat', 'p_value']].apply(pd.to_numeric)

res = multipletests(pvals=z_test_retention_conv_res.p_value, alpha=0.05, method='hs')
print(res[0], res[1])
z_test_retention_conv_res['adjusted_p'] = res[1]
z_test_retention_conv_res['reject'] = res[0]
z_test_retention_conv_res

# Группа 2 значимо отличается, возвращаемость пользователей из второй группы ниже, чем в других группах,
# размер эффекта ~6.5-7 %

In [ ]:
# Объедининяем все метрики в 1 датасет

metrics = arpu_per_group.merge(
    CR_per_group,
    how='inner',
    left_on=arpu_per_group.index,
    right_on=CR_per_group.index
).merge(
    retention_1_7_30,
    how='inner',
    left_on='key_0',
    right_on=retention_1_7_30.index
).merge(
    retention_conv_1_7_30,
    how='inner',
    left_on='key_0',
    right_on=retention_conv_1_7_30.index,
    suffixes=[None, '_paying']
)
metrics.rename(columns={'key_0':'group', 'sum':'revenue', }, inplace=True)
metrics

In [ ]:
# Выгружаем датасеты
#
# metrics.to_csv('metrics.csv')
# arpu_test.to_csv('arpu_test.csv')
# CR_z_tets.to_csv('cr_z_test.csv')
# z_test_retention_res.to_csv('z_test_retention_.csv')
# z_test_retention_conv_res.to_csv('z_test_retention_conv.csv')
# first_logins_payments.to_csv('first_logins_payments.csv')

In [ ]:
first_logins_payments.merge(
    logins[['uset_id', 'datetime']],
    how='left',
    ob='user_id'
)

# Итоги:

**ARPU**: разница ARPU всех трех групп статистически значима. Лучший результат показала группа 3, худший - группа 2.
Для второй группы на ~ $0.75-$0.86 меньше на пользователя в среднем

**CR**: Вторая группа значимо лучше конвертирует в оплату, размер эффекта ~ 0,4%. Т.е. разница в 4 человека с 1000

**Retention 30 all**: Значимой разницы нет

**Retention 30 paying users**: Группа 2 значимо отличается, возвращаемость пользователей из второй группы ниже,
чем в других группах,размер эффекта ~6.5-7 %

# Выводы:
Снижение порога входа для покупок не принесло желаемого эффекта -
для группы 2 с более дешевым предложением несмотря на больший объем изначальной выборки,
и более высокую конверсию (~ 0,4%),
средний доход на пользователя ниже практически на четверть ($0.24-$0.27) - примерно на размер уменьшения цены,
а возвращаемость на 30 день ниже на 6.5-7 %.
Меньшая цена не привлекает значимое количество пользователей, чтобы компенсировать убыток от уменьшения цены.
В дальнейшем эти пользователи не склонные тратить больше денег и меньше привязываются к игре.

Группа 3 от контрольной значимо отличается только по среднему доходу, но всего на $0,03,
что на большом объеме за 30 дней дало значительную разницу в $3719,76.
По всем остальным показателям - конверсия, возвращаемость, группа 3 превышает показатели контрольной группы,
хотя и не значимо.
На пользователей 3 группы могут влиять два фактора -
возможность купить ресурсы отдельно и длительность получения покупки.

Я бы советовала выделить эти факторы из 3 группы и протестировать отдельно,
чтобы выяснить: приводит ли какой-то из них к улучшению метрик.

